# simota

> An implementation of SimOTA label assignment for the [YOLOX](https://arxiv.org/abs/2107.08430) object detection model based on [OpenMMLab](https://github.com/open-mmlab)’s implementation in the [mmdetection](https://github.com/open-mmlab/mmdetection) library.

In [ ]:
#| default_exp simota

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass, field
from typing import Any, Type, List, Optional, Callable, Tuple, Union
from functools import partial

In [ ]:
#| export
import torch
import torch.nn.functional as F
import torchvision

In [ ]:
#| export
@dataclass
class AssignResult:
    """
    Stores assignments between predicted bounding boxes and actual truth bounding boxes.
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/core/bbox/assigners/assign_result.py#L7)

    """
    num_ground_truth_boxes: int # The number of actual truth boxes considered when computing this assignment
    ground_truth_box_indices: torch.LongTensor # For each predicted bounding box, this indicates the 1-based index of the assigned actual truth box. 0 means unassigned and -1 means ignore.
    max_iou_values: torch.FloatTensor # The Intersection over Union (IoU) between the predicted bounding box and its assigned actual truth box.
    category_labels: torch.LongTensor = field(default=None) # If specified, for each predicted bounding box, this indicates the category label of the assigned actual truth box.

In [ ]:
#| export
class SimOTAAssigner():
    """
    The `SimOTAAssigner` class assigns predicted bounding boxes to their corresponding ground truth boxes in object detection tasks. 
    It uses a process called SimOTA that formulates the assignment task as an optimal transport problem via a dynamic top-k strategy.
    
    It calculates a cost matrix based on classification and regression (Intersection over Union, IoU) costs. 
    It then uses this cost matrix to dynamically assign each ground truth object to the best matching bounding box predictions while resolving conflicts to ensure each prediction pairs with a single ground truth.
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/core/bbox/assigners/sim_ota_assigner.py#L14)
    
    """

    def __init__(self,
                 center_radius:float=2.5, # Ground truth center size to judge whether a output_grid_box is in center.
                 candidate_topk:int=10, # The candidate top-k which used to get top-k ious to calculate dynamic-k.
                 iou_weight:float=3.0, # The scale factor for regression iou cost.
                 cls_weight:float=1.0 # The scale factor for classification cost.
                ):
        self.center_radius = center_radius
        self.candidate_topk = candidate_topk
        self.iou_weight = iou_weight
        self.cls_weight = cls_weight

    def assign(self,
               pred_scores:torch.Tensor, # Classification scores of each output grid box across all classes.
               output_grid_boxes:torch.Tensor, # Output grid bounding boxes of one image in format [cx, xy, stride_w, stride_y].
               decoded_bboxes:torch.Tensor, # Predicted bounding boxes of one image in format [tl_x, tl_y, br_x, br_y].
               gt_bboxes:torch.Tensor, # Ground truth bounding boxes of one image in format [tl_x, tl_y, br_x, br_y].
               gt_labels:torch.Tensor, # Ground truth labels of one image, It is a Tensor with shape [num_gts].
               gt_bboxes_ignore:Optional[torch.Tensor]=None, # Ground truth bounding boxes that are labelled as `ignored`, e.g., crowd boxes in COCO.
               eps:float=1e-7 # A value added to the denominator for numerical stability.
              ):
        """Assign ground truth to output_grid_boxes using SimOTA.

        This method assigns predicted bounding boxes to ground truth boxes based on the computed cost matrix. 
        It first extracts valid box predictions and scores. 
        It then calculates the total cost matrix using IoU and classification costs. 
        Finally, it uses the cost matrix to assign each prediction to a ground truth box.
        """
        HIGH_COST_VALUE = 100000000
        num_gt = gt_bboxes.size(0)
        num_bboxes = decoded_bboxes.size(0)

        # assign 0 by default
        assigned_gt_inds = decoded_bboxes.new_full((num_bboxes, ), 0, dtype=torch.long)
        if num_gt == 0 or num_bboxes == 0:
            # No ground truth or boxes, return empty assignment
            max_overlaps = decoded_bboxes.new_zeros((num_bboxes, ))
            if num_gt == 0:
                # No truth, assign everything to background
                assigned_gt_inds[:] = 0
            if gt_labels is None:
                assigned_labels = None
            else:
                assigned_labels = decoded_bboxes.new_full((num_bboxes, ), -1, dtype=torch.long)
            return AssignResult(num_gt, assigned_gt_inds, max_overlaps, category_labels=assigned_labels)
        
        # Get info whether a output_grid_box is in gt bounding box and also the center of gt bounding box
        valid_mask, is_in_boxes_and_center = self.get_in_gt_and_in_center_info(output_grid_boxes, gt_bboxes)
        
        # Extract valid bounding boxes and scores (i.e., those in ground truth boxes and centers)
        valid_decoded_bbox = decoded_bboxes[valid_mask]
        valid_pred_scores = pred_scores[valid_mask]
        num_valid = valid_decoded_bbox.size(0)

        # Compute IoU between valid decoded bounding boxes and gt bounding boxes
        pairwise_ious = torchvision.ops.generalized_box_iou(valid_decoded_bbox, gt_bboxes)
            
        # Compute IoU cost
        iou_cost = -torch.log(pairwise_ious + eps)
                
        # Convert gt_labels to one-hot format and calculate classification cost
        gt_onehot_label = F.one_hot(gt_labels.to(torch.int64), pred_scores.shape[-1]).float().unsqueeze(0).repeat(num_valid, 1, 1)
        valid_pred_scores = valid_pred_scores.unsqueeze(1).repeat(1, num_gt, 1)
            
        valid_pred_scores = torch.sigmoid(valid_pred_scores)
        cls_cost = F.binary_cross_entropy(valid_pred_scores, gt_onehot_label, reduction='none').sum(-1)
#         cls_cost = F.binary_cross_entropy(valid_pred_scores.sqrt_(), gt_onehot_label, reduction='none').sum(-1)
        
        # Calculate total cost matrix by combining classification and IoU costs, 
        # and assign a high cost (HIGH_COST_VALUE) for bboxes not in both boxes and centers
        cost_matrix = cls_cost * self.cls_weight + iou_cost * self.iou_weight + (~is_in_boxes_and_center) * HIGH_COST_VALUE

        # Perform matching between ground truth and valid bounding boxes based on the cost matrix
        matched_pred_ious, matched_gt_inds = self.dynamic_k_matching(cost_matrix, pairwise_ious, num_gt, valid_mask)
        
        # Convert to AssignResult format: assign matched gt indices, labels and IoU scores
        assigned_gt_inds[valid_mask] = matched_gt_inds + 1
        assigned_labels = assigned_gt_inds.new_full((num_bboxes, ), -1)
        assigned_labels[valid_mask] = gt_labels[matched_gt_inds].long()
        max_overlaps = assigned_gt_inds.new_full((num_bboxes, ), -HIGH_COST_VALUE, dtype=torch.float32)
        max_overlaps[valid_mask] = matched_pred_ious
                    
        return AssignResult(num_gt, assigned_gt_inds, max_overlaps, category_labels=assigned_labels)

    def get_in_gt_and_in_center_info(self, 
                                     output_grid_boxes:torch.Tensor, # All output_grid_boxes of one image, a 2D-Tensor with shape [num_output_grid_boxes, 4] in [cx, xy, stride_w, stride_y] format.
                                     gt_bboxes # Ground truth bboxes of one image, a 2D-Tensor with shape [num_gts, 4] in [tl_x, tl_y, br_x, br_y] format.
                                    ) -> Tuple[torch.Tensor, torch.Tensor]: # The first tensor indicates if the output_grid_box is in any ground truth box or center, the second tensor specifies if the output_grid_box is in both the ground truth box and center.
        """Get the information about whether output_grid_boxes are in ground truth boxes or center.
        
        This method determines which predicted boxes are inside a ground truth box and also at the center of the ground truth box. 
        It computes the centers of the ground truth boxes, checks if the predicted boxes are inside the ground truth boxes and centers, 
        and then returns a mask indicating which predicted boxes are in either any ground truth box or any center box and which are in both.
        """

        # Calculate the centers of the ground truth boxes
        gt_cxs = (gt_bboxes[:, 0] + gt_bboxes[:, 2]) / 2.0
        gt_cys = (gt_bboxes[:, 1] + gt_bboxes[:, 3]) / 2.0

        # Calculate the boundaries for the ground truth boxes
        gt_bounds = torch.stack([
            output_grid_boxes[:, 0, None] - gt_bboxes[:, 0], 
            output_grid_boxes[:, 1, None] - gt_bboxes[:, 1], 
            gt_bboxes[:, 2] - output_grid_boxes[:, 0, None], 
            gt_bboxes[:, 3] - output_grid_boxes[:, 1, None]
        ], dim=1)

        # Check if output_grid_boxes are inside the ground truth boxes
        is_in_gts = gt_bounds.min(dim=1).values > 0
        is_in_gts_all = is_in_gts.any(dim=1)

        # Prepare the boundaries for the center boxes
        ct_bounds = torch.stack([
            output_grid_boxes[:, 0, None] - (gt_cxs - self.center_radius * output_grid_boxes[:, 2, None]),
            output_grid_boxes[:, 1, None] - (gt_cys - self.center_radius * output_grid_boxes[:, 3, None]),
            (gt_cxs + self.center_radius * output_grid_boxes[:, 2, None]) - output_grid_boxes[:, 0, None],
            (gt_cys + self.center_radius * output_grid_boxes[:, 3, None]) - output_grid_boxes[:, 1, None]
        ], dim=1)

        # Check if output_grid_boxes are inside the center boxes
        is_in_cts = ct_bounds.min(dim=1).values > 0
        is_in_cts_all = is_in_cts.any(dim=1)

        # Check if output_grid_boxes are in either any ground truth box or any center box
        is_in_gts_or_centers = is_in_gts_all | is_in_cts_all

        # Check if output_grid_boxes are in both ground truth boxes and centers
        is_in_boxes_and_centers = is_in_gts[is_in_gts_or_centers, :] & is_in_cts[is_in_gts_or_centers, :]

        return is_in_gts_or_centers, is_in_boxes_and_centers
    
    def dynamic_k_matching(self, 
                           cost:torch.Tensor, # A 2D tensor representing the cost matrix calculated from both classification cost and regression IoU cost. Shape is [num_output_grid_boxes, num_gts].
                           pairwise_ious:torch.Tensor, # A 2D tensor representing IoU scores between predictions and ground truths. Shape is [num_output_grid_boxes, num_gts].
                           num_gt:int, # The number of ground truth boxes.
                           valid_mask:torch.Tensor # A 1D tensor representing which predicted boxes are valid based on being in gt bboxes and in centers. Shape is [num_output_grid_boxes].
                          ) -> Tuple[torch.Tensor, torch.Tensor]: # (IoU scores for matched pairs, The indices of the ground truth for each output_grid_box)
        """
        This method performs the dynamic k-matching process. 
        For each ground truth box, it finds the top-k matching box predictions based on the smallest cost. 
        If a predicted box matches multiple ground truths, it keeps only the one with the smallest cost. 
        Finally, it returns the matched ground-truth indices and IoUs for valid predicted boxes.
        """

        # Initialize the matching matrix with zeros
        matching_matrix = torch.zeros_like(cost)

        # Select the top k IoUs for dynamic-k calculation
        topk_ious, _ = torch.topk(pairwise_ious, self.candidate_topk, dim=0)

        # Calculate dynamic k for each ground truth
        dynamic_ks = topk_ious.sum(0).int().clamp(min=1)

        # For each ground truth, find top k matching output_grid_boxes based on smallest cost
        _, pos_idx = cost.topk(k=dynamic_ks.max().item(), dim=0, largest=False)
        for gt_idx in range(num_gt):
            matching_matrix[pos_idx[:dynamic_ks[gt_idx], gt_idx], gt_idx] = 1

        # If a output_grid_box matches multiple ground truths, keep only the one with smallest cost
        output_grid_box_match_gt_mask = matching_matrix.sum(1) > 1
        if output_grid_box_match_gt_mask.any():
            _, cost_argmin = cost[output_grid_box_match_gt_mask].min(dim=1)
            matching_matrix[output_grid_box_match_gt_mask].zero_()
            matching_matrix[output_grid_box_match_gt_mask, cost_argmin] = 1

        # Update the valid mask based on final matches
        valid_mask[valid_mask.clone()] = matching_matrix.sum(1) > 0

        # Get the final matched ground truth indices and IoUs for valid predicted boxes
        fg_mask_inboxes = matching_matrix.sum(1) > 0
        matched_gt_inds = matching_matrix[fg_mask_inboxes].argmax(1)
        matched_pred_ious = (matching_matrix * pairwise_ious).sum(1)[fg_mask_inboxes]

        return matched_pred_ious, matched_gt_inds

In [ ]:
show_doc(SimOTAAssigner.assign)

---

[source](https://github.com/cj-mills/cjm-yolox-pytorch/blob/main/cjm_yolox_pytorch/simota.py#L58){target="_blank" style="float:right; font-size:smaller"}

### SimOTAAssigner.assign

>      SimOTAAssigner.assign (pred_scores:torch.Tensor,
>                             output_grid_boxes:torch.Tensor,
>                             decoded_bboxes:torch.Tensor,
>                             gt_bboxes:torch.Tensor, gt_labels:torch.Tensor,
>                             gt_bboxes_ignore:Optional[torch.Tensor]=None,
>                             eps:float=1e-07)

Assign ground truth to output_grid_boxes using SimOTA.

This method assigns predicted bounding boxes to ground truth boxes based on the computed cost matrix. 
It first extracts valid box predictions and scores. 
It then calculates the total cost matrix using IoU and classification costs. 
Finally, it uses the cost matrix to assign each prediction to a ground truth box.

In [ ]:
show_doc(SimOTAAssigner.get_in_gt_and_in_center_info)

---

[source](https://github.com/cj-mills/cjm-yolox-pytorch/blob/main/cjm_yolox_pytorch/simota.py#L130){target="_blank" style="float:right; font-size:smaller"}

### SimOTAAssigner.get_in_gt_and_in_center_info

>      SimOTAAssigner.get_in_gt_and_in_center_info
>                                                   (output_grid_boxes:torch.Ten
>                                                   sor, gt_bboxes)

Get the information about whether output_grid_boxes are in ground truth boxes or center.

This method determines which predicted boxes are inside a ground truth box and also at the center of the ground truth box. 
It computes the centers of the ground truth boxes, checks if the predicted boxes are inside the ground truth boxes and centers, 
and then returns a mask indicating which predicted boxes are in either any ground truth box or any center box and which are in both.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| output_grid_boxes | Tensor | All output_grid_boxes of one image, a 2D-Tensor with shape [num_output_grid_boxes, 4] in [cx, xy, stride_w, stride_y] format. |
| gt_bboxes |  | Ground truth bboxes of one image, a 2D-Tensor with shape [num_gts, 4] in [tl_x, tl_y, br_x, br_y] format. |
| **Returns** | **Tuple** | **The first tensor indicates if the output_grid_box is in any ground truth box or center, the second tensor specifies if the output_grid_box is in both the ground truth box and center.** |

In [ ]:
show_doc(SimOTAAssigner.dynamic_k_matching)

---

[source](https://github.com/cj-mills/cjm-yolox-pytorch/blob/main/cjm_yolox_pytorch/simota.py#L177){target="_blank" style="float:right; font-size:smaller"}

### SimOTAAssigner.dynamic_k_matching

>      SimOTAAssigner.dynamic_k_matching (cost:torch.Tensor,
>                                         pairwise_ious:torch.Tensor,
>                                         num_gt:int, valid_mask:torch.Tensor)

This method performs the dynamic k-matching process. 
For each ground truth box, it finds the top-k matching box predictions based on the smallest cost. 
If a predicted box matches multiple ground truths, it keeps only the one with the smallest cost. 
Finally, it returns the matched ground-truth indices and IoUs for valid predicted boxes.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| cost | Tensor | A 2D tensor representing the cost matrix calculated from both classification cost and regression IoU cost. Shape is [num_output_grid_boxes, num_gts]. |
| pairwise_ious | Tensor | A 2D tensor representing IoU scores between predictions and ground truths. Shape is [num_output_grid_boxes, num_gts]. |
| num_gt | int | The number of ground truth boxes. |
| valid_mask | Tensor | A 1D tensor representing which predicted boxes are valid based on being in gt bboxes and in centers. Shape is [num_output_grid_boxes]. |
| **Returns** | **Tuple** | **(IoU scores for matched pairs, The indices of the ground truth for each output_grid_box)** |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()